In [2]:
import mysql.connector

# Populate DiseaseRiskFactors table

In [4]:
# Connect to the HealthAnalysis database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123456789!",
    database="HealthAnalysis_DB"
)
cursor = conn.cursor()

# Sample data for the DiseaseRiskFactors table based on Conditions
disease_risk_factors = [
    ('Obesity', 'Poor diet', 'High'),
    ('Obesity', 'Sedentary lifestyle', 'High'),
    ('Hypertension', 'High sodium intake', 'High'),
    ('Hypertension', 'High alcohol consumption', 'Moderate'),
    ('Type 2 Diabetes', 'High sugar consumption', 'High'),
    ('Type 2 Diabetes', 'Sedentary lifestyle', 'High'),
    ('Cardiovascular Disease', 'High stress', 'High'),
    ('Cardiovascular Disease', 'High saturated fat intake', 'Moderate'),
    ('Depression', 'High alcohol consumption', 'Moderate'),
    ('Depression', 'High stress', 'High'),
    ('Anxiety', 'High caffeine consumption', 'Moderate'),
    ('Insomnia', 'Lack of sleep', 'High'),
    ('Insomnia', 'High caffeine consumption', 'Moderate'),
    ('Osteoporosis', 'Low calcium intake', 'High'),
    ('Osteoporosis', 'Low exercise frequency', 'Moderate'),
    ('Chronic Stress', 'High caffeine consumption', 'High'),
    ('Chronic Stress', 'Poor diet', 'Moderate'),
]

# Step 1: Insert data into the DiseaseRiskFactors table
for disease_name, risk_factor, risk_level in disease_risk_factors:
    sql = """INSERT INTO DiseaseRiskFactors (disease_name, risk_factor, risk_level) 
             VALUES (%s, %s, %s)"""
    values = (disease_name, risk_factor, risk_level)
    
    cursor.execute(sql, values)

# Commit the changes and close the connection
conn.commit()
cursor.close()
conn.close()

print("Disease risk factors populated successfully!")

Disease risk factors populated successfully!


# Populate PredictedIllnesses table using trained AI model

In [ ]:
import pandas as pd
import numpy as np
import random
import mysql.connector
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


C:\Users\ACER\AppData\Local\Temp\ipykernel_29048\1824330187.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123456789!",
    database="User_DB"
)
cursor = conn.cursor()

# Fetch health metrics
cursor.execute("""
    SELECT health_metric_id, user_id, avg_exercise_per_week, avg_sleep_hours, stress_level, diet_quality, most_freq_substance_used
    FROM HealthMetrics
""")
data = cursor.fetchall()

# Convert to Pandas DataFrame
df = pd.DataFrame(data, columns=["health_metric_id", "user_id", "avg_exercise_per_week", "avg_sleep_hours", "stress_level", "diet_quality", "most_freq_substance_used"])

# Close connection
cursor.close()
conn.close()


In [3]:
# Encoding categorical variables
df["diet_quality"] = df["diet_quality"].map({"Poor": 1, "Average": 2, "Good": 3, "Very good": 4})
df["most_freq_substance_used"] = df["most_freq_substance_used"].map({"None": 0, "Caffeine": 1, "Alcohol": 2, "Tobacco": 3})

# Generating synthetic illness labels based on risk factors
def assign_illness(row):
    if row["stress_level"] >= 8 and row["avg_sleep_hours"] < 6:
        return "Hypertension"
    elif row["most_freq_substance_used"] == 3 and row["diet_quality"] == 1:
        return "Lung Disease"
    elif row["avg_exercise_per_week"] < 2 and row["diet_quality"] in [1, 2]:
        return "Obesity"
    else:
        return "No Risk"

df["predicted_disease"] = df.apply(assign_illness, axis=1)

# Drop users with "No Risk" if you only want to predict for at-risk users
df = df[df["predicted_disease"] != "No Risk"]

# Define features and labels
X = df.drop(columns=["predicted_disease", "health_metric_id", "user_id"])  # Features
y = df["predicted_disease"]  # Labels


In [4]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 100.00%


In [5]:
# Reconnect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123456789!",
    database="HealthAnalysis_DB"
)
cursor = conn.cursor()

# Predict illness for all users
df["predicted_disease"] = model.predict(X)
df["confidence_score"] = np.max(model.predict_proba(X), axis=1)  # Confidence score
df["risk_level"] = df["confidence_score"].apply(lambda x: "High" if x > 0.8 else "Moderate" if x > 0.5 else "Low")

# Insert predictions into PredictedIllnesses table
for index, row in df.iterrows():
    sql = """INSERT INTO PredictedIllnesses (user_id, health_metric_id, predicted_disease, confidence_score, risk_level)
             VALUES (%s, %s, %s, %s, %s)"""
    values = (row["user_id"], row["health_metric_id"], row["predicted_disease"], row["confidence_score"], row["risk_level"])
    
    cursor.execute(sql, values)

# Commit changes
conn.commit()

# Close connection
cursor.close()
conn.close()

print(f"{len(df)} illness predictions inserted into PredictedIllnesses table.")


70 illness predictions inserted into PredictedIllnesses table.


## To predict new user input

In [12]:
# Function to store predictions into the database
def store_data_into_db(user_id, health_metric_id, predicted_disease, confidence_score, risk_level):
    try:
        # Convert confidence_score to float
        confidence_score = float(confidence_score)

        # Connect to MySQL
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="123456789!",
            database="HealthAnalysis_DB"
        )
        cursor = conn.cursor()

        # Insert prediction into PredictedIllnesses table
        insert_query = """
        INSERT INTO PredictedIllnesses (user_id, health_metric_id, predicted_disease, confidence_score, risk_level)
        VALUES (%s, %s, %s, %s, %s)
        """
        values = (user_id, health_metric_id, predicted_disease, confidence_score, risk_level)

        cursor.execute(insert_query, values)
        conn.commit()

        print(f"Prediction for User {user_id} stored successfully!")

    except mysql.connector.Error as err:
        print(f"Error: {err}")
    
    finally:
        cursor.close()
        conn.close()

# Function to predict illness and store it in the database
def predict_new_user_illness(new_user_data, model):
    # Convert input to DataFrame
    new_user_df = pd.DataFrame([new_user_data])

    # Apply encoding
    new_user_df["diet_quality"] = new_user_df["diet_quality"].map({"Poor": 1, "Average": 2, "Good": 3, "Very good": 4})
    new_user_df["most_freq_substance_used"] = new_user_df["most_freq_substance_used"].map({"None": 0, "Caffeine": 1, "Alcohol": 2, "Tobacco": 3})

    # Ensure correct feature order
    new_user_X = new_user_df[["avg_exercise_per_week", "avg_sleep_hours", "stress_level", "diet_quality", "most_freq_substance_used"]]

    # Make prediction
    predicted_disease = model.predict(new_user_X)[0]
    confidence_score = np.max(model.predict_proba(new_user_X), axis=1)[0]
    risk_level = "High" if confidence_score > 0.8 else "Moderate" if confidence_score > 0.5 else "Low"

    # Store prediction in the database
    store_data_into_db(new_user_data["user_id"], new_user_data["health_metric_id"], predicted_disease, confidence_score, risk_level)

    return {
        "predicted_disease": predicted_disease,
        "confidence_score": confidence_score,
        "risk_level": risk_level
    }

# Define new user data, including health_metric_id
new_user = {
    "user_id": 101,  # Replace with actual user ID
    "health_metric_id": 1,  # Replace with the correct health_metric_id
    "avg_exercise_per_week": 1,
    "avg_sleep_hours": 5,
    "stress_level": 9,
    "diet_quality": "Poor",
    "most_freq_substance_used": "Tobacco"
}

# Predict and store result
prediction_result = predict_new_user_illness(new_user, model)
print("New User Prediction:", prediction_result)


Prediction for User 101 stored successfully!
New User Prediction: {'predicted_disease': 'Hypertension', 'confidence_score': 0.86, 'risk_level': 'High'}
